In [8]:
import torch
import os 
import sys
import umap, numba
import numpy as np
import tqdm
import glob, pandas as pd
import pickle
import fire
import scipy.sparse as sps
from visdom import Visdom
from sklearn.metrics import f1_score
import copy
import torch.nn.functional as F
from torch import nn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import torch_cluster
from torch_cluster import knn_graph
from torch_geometric.data import Data 
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data import InMemoryDataset,DataLoader
from torch_geometric.utils import add_remaining_self_loops
from torch_cluster import radius_graph
from torch_geometric.utils import subgraph

In [10]:
IDs=['109_A1c_ASAP_tumor_map', '10_A1a_ASAP_tumor_map', '10_A1b_ASAP_tumor_map', '10_A2b_ASAP_tumor_map', '110_A2b_ASAP_tumor_map', '112_a_ASAP_tumor_map', '112_b_ASAP_tumor_map', '123_A1a_ASAP_tumor_map', '12_A1c_ASAP_tumor_map', '14_A1b_ASAP_tumor_map', '14_A2b_ASAP_tumor_map', '169_A2b_ASAP_tumor_map', '270_A1b_ASAP_tumor_map', '270_A1d_ASAP_tumor_map', '270_A1e_ASAP_tumor_map', '270_A2b_ASAP_tumor_map', '270_A2f_ASAP_tumor_map', '281_A1d_ASAP_tumor_map', '281_A1f_ASAP_tumor_map', '281_A2eX_ASAP_tumor_map', '311_A2c_ASAP_tumor_map', '327_A1a_ASAP_tumor_map', '327_A1d_ASAP_tumor_map', '327_B1c_ASAP_tumor_map', '341_a_ASAP_tumor_map', '341_b_ASAP_tumor_map', '342_a_ASAP_tumor_map', '342_b_ASAP_tumor_map', '343_a_ASAP_tumor_map', '343_b_ASAP_tumor_map', '343_c_ASAP_tumor_map', '343_d_ASAP_tumor_map', '344_a_ASAP_tumor_map', '344_b_ASAP_tumor_map', '345_a_ASAP_tumor_map', '345_b_ASAP_tumor_map', '346_a_ASAP_tumor_map', '346_b_ASAP_tumor_map', '350_A1a_ASAP_tumor_map', '350_A1b_ASAP_tumor_map', '350_A1c_ASAP_tumor_map', '350_A1d_ASAP_tumor_map', '350_A1e_ASAP_tumor_map', '351_A2b_ASAP_tumor_map', '352_A1d_ASAP_tumor_map', '352_A1e_ASAP_tumor_map', '352_A1g_ASAP_tumor_map', '352_A1h_ASAP_tumor_map', '352_A1i_ASAP_tumor_map', '353_A2b_ASAP_tumor_map', '354_A1b_ASAP_tumor_map', '354_A1c_ASAP_tumor_map', '354_A1d_ASAP_tumor_map', '354_A3a_ASAP_tumor_map', '354_A3b_ASAP_tumor_map', '354_A3c_ASAP_tumor_map', '354_D1b_ASAP_tumor_map', '355_A1d_ASAP_tumor_map', '356_A1b_ASAP_tumor_map', '358_A1a_ASAP_tumor_map', '358_A1b_ASAP_tumor_map', '361_a_ASAP_tumor_map', '361_b_ASAP_tumor_map', '362_A1a_ASAP_tumor_map', '362_A1b_ASAP_tumor_map', '362_A1c_ASAP_tumor_map', '363_A1b_ASAP_tumor_map', '363_A1c_ASAP_tumor_map', '363_A2b_ASAP_tumor_map', '363_A3b_ASAP_tumor_map', '364_A1b_ASAP_tumor_map', '364_A2b_ASAP_tumor_map', '364_A4b_ASAP_tumor_map', '365_A1b_ASAP_tumor_map', '365_A2b_ASAP_tumor_map', '366_A1a_ASAP_tumor_map', '366_A1b_ASAP_tumor_map', '366_A1c_ASAP_tumor_map', '367_A2b_ASAP_tumor_map', '368_A1b_ASAP_tumor_map', '368_A1c_ASAP_tumor_map', '368_A1d_ASAP_tumor_map', '369_A1b_ASAP_tumor_map', '369_A1c_ASAP_tumor_map', '369_A2b_ASAP_tumor_map', '370_A1b_ASAP_tumor_map', '370_A2a_ASAP_tumor_map', '370_A2b_ASAP_tumor_map', '37_A2d_ASAP_tumor_map', '61_A1a_ASAP_tumor_map', '61_B1a_ASAP_tumor_map', '70_A2b_ASAP_tumor_map', '7_A1c_ASAP_tumor_map', '7_A1d_ASAP_tumor_map', '7_A1e_ASAP_tumor_map']

In [13]:

#TUMOR_PERCENT=0.2

def get_graph_dataset(embedding_dir='', 
                      basename='', 
                      target_col='scc',  
                      k=8,
                      radius=0,
                      dataset_save_loc=''):
    with open(os.path.join(embedding_dir,f"{basename}"),"rb") as file:
        embeddings=torch.load(file) 
    df=embeddings['patch_info']
    X=pd.DataFrame(embeddings['embeddings'])
    df['y_true']=df[target_col]
    xy=torch.tensor(embeddings['patch_info'][['x','y']].values)
    X=torch.tensor(X.values)
    y=torch.tensor(df.loc[:,'y_true'].values)
    
    if not radius:
        G=knn_graph(xy,k=k)
    else:
        G=radius_graph(xy, r=radius*np.sqrt(2), batch=None, loop=True)
    G=(add_remaining_self_loops(G)[0]).detach().cpu()
    xy=xy.detach().cpu()
    datasets=[]
    edges=G.detach().cpu().numpy().astype(int)
    n_components,components=list(sps.csgraph.connected_components(sps.coo_matrix((np.ones_like(edges[0]),(edges[0],edges[1])))))
    components=torch.LongTensor(components)
    for i in range(n_components):
        G_new=subgraph(components==i,G,relabel_nodes=True)[0]
        xy_new=xy[components==i]
        X_new=X[components==i]
        ohe = OneHotEncoder()
        y_new=(y[components==i]).reshape(-1,1)
        y_new=ohe.fit_transform((y_new)).toarray()
        #y_whole_slide_image_label=y_new.float().mean()>=TUMOR_PERCENT
        np.random.seed(42)
        idx=np.arange(X_new.shape[0])
        idx2=np.arange(X_new.shape[0])
        np.random.shuffle(idx)
        train_idx,val_idx,test_idx=torch.tensor(np.isin(idx2,idx[:int(0.8*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.8*len(idx)):int(0.9*len(idx))])),torch.tensor(np.isin(idx2,idx[int(0.9*len(idx)):]))
        dataset=Data(x=X_new, edge_index=G_new, edge_attr=None, y=y_new, pos=xy_new)
        dataset.train_mask=train_idx
        dataset.val_mask=val_idx
        dataset.test_mask=test_idx
        datasets.append(dataset)
    #components=components.numpy()
    
    torch.save(datasets, os.path.join(dataset_save_loc,f"dataset{ID}.pkl"))
    return datasets
    



In [12]:

get_graph_dataset(embedding_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/",
                  basename='109_A1c_ASAP_tumor_map' +'.pkl',
                  target_col='scc',
                  k=8,
                  radius=256,
                  dataset_save_loc=f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/112_a_ASAP_tumor_map.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/112_a_ASAP_tumor_map.pkl/dataset.pkl'

In [37]:
IDs=['109_A1c_ASAP_tumor_map', '10_A1a_ASAP_tumor_map', '10_A1b_ASAP_tumor_map', '10_A2b_ASAP_tumor_map', '110_A2b_ASAP_tumor_map', '112_a_ASAP_tumor_map', '112_b_ASAP_tumor_map', '123_A1a_ASAP_tumor_map', '12_A1c_ASAP_tumor_map', '14_A1b_ASAP_tumor_map', '14_A2b_ASAP_tumor_map', '169_A2b_ASAP_tumor_map', '270_A1b_ASAP_tumor_map', '270_A1d_ASAP_tumor_map', '270_A1e_ASAP_tumor_map', '270_A2b_ASAP_tumor_map', '270_A2f_ASAP_tumor_map', '281_A1d_ASAP_tumor_map', '281_A1f_ASAP_tumor_map', '281_A2eX_ASAP_tumor_map', '311_A2c_ASAP_tumor_map', '327_A1a_ASAP_tumor_map', '327_A1d_ASAP_tumor_map', '327_B1c_ASAP_tumor_map', '341_a_ASAP_tumor_map', '341_b_ASAP_tumor_map', '342_a_ASAP_tumor_map', '342_b_ASAP_tumor_map', '343_a_ASAP_tumor_map', '343_b_ASAP_tumor_map', '343_c_ASAP_tumor_map', '343_d_ASAP_tumor_map', '344_a_ASAP_tumor_map', '344_b_ASAP_tumor_map', '345_a_ASAP_tumor_map', '345_b_ASAP_tumor_map', '346_a_ASAP_tumor_map', '346_b_ASAP_tumor_map', '350_A1a_ASAP_tumor_map', '350_A1b_ASAP_tumor_map', '350_A1c_ASAP_tumor_map', '350_A1d_ASAP_tumor_map', '350_A1e_ASAP_tumor_map', '351_A2b_ASAP_tumor_map', '352_A1d_ASAP_tumor_map', '352_A1e_ASAP_tumor_map', '352_A1g_ASAP_tumor_map', '352_A1h_ASAP_tumor_map', '352_A1i_ASAP_tumor_map', '353_A2b_ASAP_tumor_map', '354_A1b_ASAP_tumor_map', '354_A1c_ASAP_tumor_map', '354_A1d_ASAP_tumor_map', '354_A3a_ASAP_tumor_map', '354_A3b_ASAP_tumor_map', '354_A3c_ASAP_tumor_map', '354_D1b_ASAP_tumor_map', '355_A1d_ASAP_tumor_map', '356_A1b_ASAP_tumor_map', '358_A1a_ASAP_tumor_map', '358_A1b_ASAP_tumor_map', '361_a_ASAP_tumor_map', '361_b_ASAP_tumor_map', '362_A1a_ASAP_tumor_map', '362_A1b_ASAP_tumor_map', '362_A1c_ASAP_tumor_map', '363_A1b_ASAP_tumor_map', '363_A1c_ASAP_tumor_map', '363_A2b_ASAP_tumor_map', '363_A3b_ASAP_tumor_map', '364_A1b_ASAP_tumor_map', '364_A2b_ASAP_tumor_map', '364_A4b_ASAP_tumor_map', '365_A1b_ASAP_tumor_map', '365_A2b_ASAP_tumor_map', '366_A1a_ASAP_tumor_map', '366_A1b_ASAP_tumor_map', '366_A1c_ASAP_tumor_map', '367_A2b_ASAP_tumor_map', '368_A1b_ASAP_tumor_map', '368_A1c_ASAP_tumor_map', '368_A1d_ASAP_tumor_map', '369_A1b_ASAP_tumor_map', '369_A1c_ASAP_tumor_map', '369_A2b_ASAP_tumor_map', '370_A1b_ASAP_tumor_map', '370_A2a_ASAP_tumor_map', '370_A2b_ASAP_tumor_map', '37_A2d_ASAP_tumor_map', '61_A1a_ASAP_tumor_map', '61_B1a_ASAP_tumor_map', '70_A2b_ASAP_tumor_map', '7_A1c_ASAP_tumor_map', '7_A1d_ASAP_tumor_map', '7_A1e_ASAP_tumor_map']

for ID in tqdm.tqdm(IDs):
    get_graph_dataset(embedding_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/",
                  basename=ID +'.pkl',
                  target_col='scc',
                  k=8,
                  radius=256)
    


100%|██████████████████████████████████████████████████████████████████████████| 95/95 [01:32<00:00,  1.03it/s]


In [40]:
IDs=['109_A1c_ASAP_tumor_map', '10_A1a_ASAP_tumor_map', '10_A1b_ASAP_tumor_map', '10_A2b_ASAP_tumor_map', '110_A2b_ASAP_tumor_map', '112_a_ASAP_tumor_map', '112_b_ASAP_tumor_map', '123_A1a_ASAP_tumor_map', '12_A1c_ASAP_tumor_map', '14_A1b_ASAP_tumor_map', '14_A2b_ASAP_tumor_map', '169_A2b_ASAP_tumor_map', '270_A1b_ASAP_tumor_map', '270_A1d_ASAP_tumor_map', '270_A1e_ASAP_tumor_map', '270_A2b_ASAP_tumor_map', '270_A2f_ASAP_tumor_map', '281_A1d_ASAP_tumor_map', '281_A1f_ASAP_tumor_map', '281_A2eX_ASAP_tumor_map', '311_A2c_ASAP_tumor_map', '327_A1a_ASAP_tumor_map', '327_A1d_ASAP_tumor_map', '327_B1c_ASAP_tumor_map', '341_a_ASAP_tumor_map', '341_b_ASAP_tumor_map', '342_a_ASAP_tumor_map', '342_b_ASAP_tumor_map', '343_a_ASAP_tumor_map', '343_b_ASAP_tumor_map', '343_c_ASAP_tumor_map', '343_d_ASAP_tumor_map', '344_a_ASAP_tumor_map', '344_b_ASAP_tumor_map', '345_a_ASAP_tumor_map', '345_b_ASAP_tumor_map', '346_a_ASAP_tumor_map', '346_b_ASAP_tumor_map', '350_A1a_ASAP_tumor_map', '350_A1b_ASAP_tumor_map', '350_A1c_ASAP_tumor_map', '350_A1d_ASAP_tumor_map', '350_A1e_ASAP_tumor_map', '351_A2b_ASAP_tumor_map', '352_A1d_ASAP_tumor_map', '352_A1e_ASAP_tumor_map', '352_A1g_ASAP_tumor_map', '352_A1h_ASAP_tumor_map', '352_A1i_ASAP_tumor_map', '353_A2b_ASAP_tumor_map', '354_A1b_ASAP_tumor_map', '354_A1c_ASAP_tumor_map', '354_A1d_ASAP_tumor_map', '354_A3a_ASAP_tumor_map', '354_A3b_ASAP_tumor_map', '354_A3c_ASAP_tumor_map', '354_D1b_ASAP_tumor_map', '355_A1d_ASAP_tumor_map', '356_A1b_ASAP_tumor_map', '358_A1a_ASAP_tumor_map', '358_A1b_ASAP_tumor_map', '361_a_ASAP_tumor_map', '361_b_ASAP_tumor_map', '362_A1a_ASAP_tumor_map', '362_A1b_ASAP_tumor_map', '362_A1c_ASAP_tumor_map', '363_A1b_ASAP_tumor_map', '363_A1c_ASAP_tumor_map', '363_A2b_ASAP_tumor_map', '363_A3b_ASAP_tumor_map', '364_A1b_ASAP_tumor_map', '364_A2b_ASAP_tumor_map', '364_A4b_ASAP_tumor_map', '365_A1b_ASAP_tumor_map', '365_A2b_ASAP_tumor_map', '366_A1a_ASAP_tumor_map', '366_A1b_ASAP_tumor_map', '366_A1c_ASAP_tumor_map', '367_A2b_ASAP_tumor_map', '368_A1b_ASAP_tumor_map', '368_A1c_ASAP_tumor_map', '368_A1d_ASAP_tumor_map', '369_A1b_ASAP_tumor_map', '369_A1c_ASAP_tumor_map', '369_A2b_ASAP_tumor_map', '370_A1b_ASAP_tumor_map', '370_A2a_ASAP_tumor_map', '370_A2b_ASAP_tumor_map', '37_A2d_ASAP_tumor_map', '61_A1a_ASAP_tumor_map', '61_B1a_ASAP_tumor_map', '70_A2b_ASAP_tumor_map', '7_A1c_ASAP_tumor_map', '7_A1d_ASAP_tumor_map', '7_A1e_ASAP_tumor_map']

all_dataset=[]
for ID in (IDs):
    graph_dataset=get_graph_dataset(embedding_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/",
                  basename=ID +'.pkl',
                  target_col='scc',
                  k=8,
                  radius=256)
    all_dataset.append(graph_dataset)

[Data(x=[4639, 2048], edge_index=[2, 22765], y=[4639], pos=[4639, 2], train_mask=[4639], val_mask=[4639], test_mask=[4639]),
 Data(x=[4710, 2048], edge_index=[2, 23116], y=[4710], pos=[4710, 2], train_mask=[4710], val_mask=[4710], test_mask=[4710]),
 Data(x=[5083, 2048], edge_index=[2, 24985], y=[5083], pos=[5083, 2], train_mask=[5083], val_mask=[5083], test_mask=[5083])]

In [ ]:
import pickle
pickle.dump(dict(graph_dataset=graph_datasets,train_test_split))

In [13]:
df=pd.read_pickle("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/file_locations.pkl")

In [14]:
import math
from sklearn.model_selection import train_test_split
from itertools import chain
graph_datasets={}
for ID in tqdm.tqdm(IDs):
    all_dataset = get_graph_dataset(embedding_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/", 
                                    basename=ID+ ".pkl", 
                                    target_col='scc', 
                                    k=8, 
                                    radius=256, 
                                    dataset_save_loc=f"/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/")                        
    print(ID, len(all_dataset))
    if len(all_dataset) == 1:
        graph_datasets[ID]= {'train' : all_dataset, 'val' : []}
    else:
        train_size=int(math.ceil(len(all_dataset)*0.9))
        train_dataset, val_dataset = train_test_split(all_dataset,train_size=0.90,random_state=42)
        graph_datasets[ID]= {'train' : train_dataset, 'val' : val_dataset}


  1%|▌                                                    | 1/95 [00:05<07:56,  5.06s/it]

109_A1c_ASAP_tumor_map 2


  2%|█                                                    | 2/95 [00:07<05:03,  3.27s/it]

10_A1a_ASAP_tumor_map 3


  3%|█▋                                                   | 3/95 [00:09<04:12,  2.74s/it]

...

7_A1c_ASAP_tumor_map 3


 99%|███████████████████████████████████████████████████▍| 94/95 [03:19<00:01,  1.82s/it]

7_A1d_ASAP_tumor_map 3


100%|████████████████████████████████████████████████████| 95/95 [03:21<00:00,  2.12s/it]

7_A1e_ASAP_tumor_map 3


In [15]:
graph_datasets

{'109_A1c_ASAP_tumor_map': {'train': [Data(x=[20589, 2048], edge_index=[2, 102219], y=[20589, 2], pos=[20589, 2], train_mask=[20589], val_mask=[20589], test_mask=[20589])],
  'val': [Data(x=[20706, 2048], edge_index=[2, 102704], y=[20706, 2], pos=[20706, 2], train_mask=[20706], val_mask=[20706], test_mask=[20706])]},
 '10_A1a_ASAP_tumor_map': {'train': [Data(x=[5563, 2048], edge_index=[2, 27309], y=[5563, 2], pos=[5563, 2], train_mask=[5563], val_mask=[5563], test_mask=[5563]),
   Data(x=[5101, 2048], edge_index=[2, 24973], y=[5101, 2], pos=[5101, 2], train_mask=[5101], val_mask=[5101], test_mask=[5101])],
  'val': [Data(x=[5804, 2048], edge_index=[2, 28532], y=[5804, 2], pos=[5804, 2], train_mask=[5804], val_mask=[5804], test_mask=[5804])]},
 '10_A1b_ASAP_tumor_map': {'train': [Data(x=[5719, 2048], edge_index=[2, 28113], y=[5719, 2], pos=[5719, 2], train_mask=[5719], val_mask=[5719], test_mask=[5719]),
   Data(x=[5649, 2048], edge_index=[2, 27761], y=[5649, 2], pos=[5649, 2], train_ma

In [16]:
import pickle
pickle.dump(graph_datasets,open('/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/graph_dataset.pkl','wb'))

In [17]:
e=pd.read_pickle("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/graph_dataset.pkl")

In [18]:
e

{'109_A1c_ASAP_tumor_map': {'train': [Data(x=[20589, 2048], edge_index=[2, 102219], y=[20589], pos=[20589, 2], train_mask=[20589], val_mask=[20589], test_mask=[20589])],
  'val': [Data(x=[20706, 2048], edge_index=[2, 102704], y=[20706], pos=[20706, 2], train_mask=[20706], val_mask=[20706], test_mask=[20706])]},
 '10_A1a_ASAP_tumor_map': {'train': [Data(x=[5563, 2048], edge_index=[2, 27309], y=[5563], pos=[5563, 2], train_mask=[5563], val_mask=[5563], test_mask=[5563]),
   Data(x=[5101, 2048], edge_index=[2, 24973], y=[5101], pos=[5101, 2], train_mask=[5101], val_mask=[5101], test_mask=[5101])],
  'val': [Data(x=[5804, 2048], edge_index=[2, 28532], y=[5804], pos=[5804, 2], train_mask=[5804], val_mask=[5804], test_mask=[5804])]},
 '10_A1b_ASAP_tumor_map': {'train': [Data(x=[5719, 2048], edge_index=[2, 28113], y=[5719], pos=[5719, 2], train_mask=[5719], val_mask=[5719], test_mask=[5719]),
   Data(x=[5649, 2048], edge_index=[2, 27761], y=[5649], pos=[5649, 2], train_mask=[5649], val_mask=[

In [19]:
!nvidia-smi

Tue Dec 20 23:37:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   41C    P0    74W / 300W |   2132MiB / 32768MiB |     42%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [2]:
import pandas as pd
import torch
e=torch.load("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/cnn_embeddings/7_A1d_ASAP_tumor_map.pkl")

In [3]:
e

{'embeddings': array([[  74.16347 ,   77.23718 ,  910.82556 , ...,   76.0359  ,
         1116.2341  , 2357.2185  ],
        [  67.3526  ,   86.756546,  881.6731  , ...,   78.82091 ,
         1090.5277  , 2317.262   ],
        [  49.52798 ,   77.0425  ,  492.2309  , ...,  114.142586,
          758.39246 , 1577.464   ],
        ...,
        [  76.5256  ,   77.83744 ,  691.3006  , ...,  100.117546,
          931.68726 , 2067.2043  ],
        [  54.594597,   48.091347,  819.1297  , ...,   79.47032 ,
          848.91907 , 2026.8959  ],
        [  56.471123,   45.852936,  893.09344 , ...,  100.63023 ,
         1033.5111  , 2287.923   ]], dtype=float32),
 'patch_info':                ID      x      y  patch_size annotation  y_true  inflamm  scc
 0      7_A1d_ASAP   1024  14848         256          0       0        0    0
 1      7_A1d_ASAP   1280  14336         256          0       0        0    0
 2      7_A1d_ASAP   1280  14592         256          0       0        0    0
 3      7_A1d_ASAP